## Generate Stateset for the day

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from custom_pipeline import get_concs, get_stateset, scale_stateset
from pathlib import Path
import sys
import pickle
package_path = Path.cwd().parent
if package_path not in sys.path:
    sys.path.append(str(package_path))
from utils.dataset_class import Phase3DataSet, Setting

In [2]:
stateset_raw = pd.read_csv('../phase3/final_data.csv')
chemical_amounts = pd.read_csv('./extra_data/al_ingredients.csv')

In [3]:
actual_amounts = {
    '_raw_reagent_0_chemicals_0_actual_amount' : 1.37, #ml    - DMF     - Reagent 1
    
    '_raw_reagent_1_chemicals_0_actual_amount' : 0.7641, #grams - PbI2    - Reagent 2
    '_raw_reagent_1_chemicals_1_actual_amount' : 0.2927, #grams - Me2NH2I - Reagent 2
    '_raw_reagent_1_chemicals_2_actual_amount' : 0.63, #ml    - DMF     - Reagent 2
    
    '_raw_reagent_2_chemicals_0_actual_amount' : 0.5222, #grams - Me2NH2I - Reagent 3
    '_raw_reagent_2_chemicals_1_actual_amount' : 1.06, #ml    - DMF     - Reagent 3
    
    '_raw_reagent_6_chemicals_0_actual_amount' : 2.42, #ml    - FAH     - Reagent 5
    
}
for chemical in actual_amounts:
    chemical_amounts.loc[0, chemical] = actual_amounts[chemical]

In [4]:
stateset_template = pd.read_csv('./extra_data/Me2NH2I_10uL_stateset.link.csv')
ss_reagent_vols = stateset_template[[col for col in stateset_template.columns if 'Reagent' in col]]
chemdf_dict = {'HC': pd.read_csv('./extra_data/chemical_inventory.csv', index_col='InChI Key (ID)')}
phase3_training = pickle.load(open('phase3_dataset.pkl', 'rb'))

In [5]:
summed_mmol, summed_molarity = get_concs(chemical_amounts, chemdf_dict, ss_reagent_vols)
summed_mmol.index.rename('name')
summed_molarity.index.rename('name')
stateset = get_stateset(summed_mmol, summed_molarity)
x_stateset = scale_stateset(stateset, phase3_training.scaler)

## Write today's stateset to disk

In [6]:
stateset.to_csv(f'./statesets/stateset_{datetime.today().date()}.csv')
np.savetxt(f'./statesets/scaled_stateset_{datetime.today().date()}.csv', x_stateset, delimiter=",")